In [1]:
import pandas as pd

In [13]:
og_breast_cancer_data = pd.read_csv('original_dataset.csv', header=None, names=["ID", "Clump_Thickness", "Cell_Size_Uniformity", "Cell_Shape_Uniformity", "Marginal_Adhesion", "Epithelial_Cell_Size", "Bare_Nuclei", "Bland_Chromatin", "Normal_Nucleoli", "Mitoses", "Class"])

In [16]:
og_breast_cancer_data['Class'] = og_breast_cancer_data['Class'].replace([2,4], [0,1])

In [ ]:
og_breast_cancer_data['Class']